In [1]:
ls

ItemBasedRecoms.csv           item-based-recommender.ipynb
Untitled.ipynb


In [1]:
import pandas as pd
import numpy as np
import ast
pd.set_option('max_colwidth', 300)

In [292]:
#functions : bunlar RecipeBox.py de diğer fonksiyonlarla birlikte ilk anda çalıştırılmalı
def remove_recipe(row):
    recom_ids = row['recom_ids']
    recom_names = row['recom_names']
    recom_corrs = row['recom_corrs']
    
    new_recom_ids,new_recom_names,new_recom_corrs = [],[],[]
    
    for id, name, corr in zip(recom_ids, recom_names, recom_corrs):
        if id not in recom_ids:
            if pd.notnull(corr) and corr > 0:
                new_recom_ids.append(id)
                new_recom_names.append(name)
                new_recom_corrs.append(corr)
    
    return new_recom_ids, new_recom_names, new_recom_corrs


def convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        return np.nan

In [302]:
item_based_df=pd.read_csv('ItemBasedRecoms.csv')

In [303]:
item_based_updated = pd.DataFrame()

for index, row in item_based_df.iterrows():
    recipe_id = row['recipe_id']
    filtered_item = item_based_df[item_based_df.recipe_id == recipe_id].copy()
    filtered_item['recom_ids'] = filtered_item['recom_ids'].str.strip("[]").str.split(", ")
    filtered_item['recom_names'] = filtered_item['recom_names'].str.strip("[]").str.split(", ")
    filtered_item['recom_corrs'] = filtered_item['recom_corrs'].str.strip("[]").str.split(", ").apply(lambda x: [convert_to_float(val) if val != 'NaN' else np.nan for val in x])

    new_recom_ids = []
    new_recom_names = []
    new_recom_corrs = []

    for ids, names, corrs in zip(filtered_item['recom_ids'], filtered_item['recom_names'], filtered_item['recom_corrs']):
        new_ids = []
        new_names = []
        new_corrs = []
        for id, name, corr in zip(ids, names, corrs):
            if not pd.isnull(corr) and float(corr) > 0 and id != recipe_id:
                new_ids.append(id)
                new_names.append(name.strip("'"))
                new_corrs.append(corr)
        new_recom_ids.append(new_ids)
        new_recom_names.append(new_names)
        new_recom_corrs.append(new_corrs)

    filtered_item['recom_ids'] = new_recom_ids
    filtered_item['recom_names'] = new_recom_names
    filtered_item['recom_corrs'] = new_recom_corrs

    recipe_dict = {}
    for ids, names in zip(filtered_item['recom_ids'], filtered_item['recom_names']):
        for id, name in zip(ids, names):
            recipe_dict[int(id)] = name

    if recipe_id in recipe_dict:
        del recipe_dict[recipe_id]

    filtered_item['recipe_dict'] = [recipe_dict] * len(filtered_item)
    item_based_updated = pd.concat([item_based_updated, filtered_item], ignore_index=True)

In [307]:
item_based_updated.head(3)

,recipe_id,recom_ids,recom_names,recom_corrs,recipe_dict
0,25775,"[103572, 57772, 49387, 128956, 73033, 90485]","[crock pot black bean salsa chicken oamc, mexican stuffed shells oamc, oven fried eggplant aubergine, ww 0 point weight watchers cabbage soup, allrighty then scrambled eggs paula deen, balsamic chicken with garlic couscous]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","{103572: 'crock pot black bean salsa chicken oamc', 57772: 'mexican stuffed shells oamc', 49387: 'oven fried eggplant aubergine', 128956: 'ww 0 point weight watchers cabbage soup', 73033: 'allrighty then scrambled eggs paula deen', 90485: 'balsamic chicken with garlic couscous'}"
1,22526,"[251220, 56846, 84104, 83524, 83506, 21688]","[kittencal s fluffiest scrambled eggs, i hate ricotta meat 2 cheese stuffed shells, spicy hasselback potatoes, frizzled cabbage, sharon s ono barbecue beef, cabbage beef casserole]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","{251220: 'kittencal s fluffiest scrambled eggs', 56846: 'i hate ricotta meat 2 cheese stuffed shells', 84104: 'spicy hasselback potatoes', 83524: 'frizzled cabbage', 83506: 'sharon s ono barbecue beef', 21688: 'cabbage beef casserole'}"
2,74805,"[34382, 150863, 110683, 3370, 207872, 111734]","[mashed red potatoes with garlic and parmesan, panera s cream cheese potato soup, basic batter waffles, french toast ii, baked seasoned salmon, easy pizza pasta casserole oamc]","[1.0, 1.0, 1.0, 0.9999999999999999, 0.9999999999999999, 0.9999999999999999]","{34382: 'mashed red potatoes with garlic and parmesan', 150863: 'panera s cream cheese potato soup', 110683: 'basic batter waffles', 3370: 'french toast ii', 207872: 'baked seasoned salmon', 111734: 'easy pizza pasta casserole oamc'}"


In [315]:
item_based_updated.to_csv('item_based_updated.csv', index=False)

In [8]:
import ast

recipe_id = 59346 #buraya önceki sayfadan tıklanan recipe_id atanmalıdır. 59346 deneme için kullanıldı

#Item based için çıkarılan 6şar öneriden, önerilerde kendisi varsa veya 
#corelasyonu NaN, negatifse elenerek yeni bir kolona(recom_dict) atandı.
#Bu yeni veri item_based_updated.csv olarak kaydedildi. Item based önerile için bu veri kullanılacaktır.
item_based_recom=pd.read_csv('item_based_updated.csv') #buradaki path proje dosyasına göre düzeltilmelidir.

filtered_recom = item_based_recom[item_based_recom.recipe_id == recipe_id]

#Öneri hiç yoksa bi yazı yazdırabiliriz. Yazıya karar verelim
if filtered_recom.recipe_dict.isnull().any(): 
    print("Bu tarifi beğenenler çok fazla tarif beğenmemiş :)")
else: #Öneri varsa idlerini ve isimlerini recom_df adında bir dataframe de tutar.
    recipe_dict = ast.literal_eval(filtered_recom.recipe_dict.iloc[0])
    recom_df = pd.DataFrame(list(recipe_dict.items()), columns=['id', 'name'])

recom_df #recom_df kullanılarak item based öneriler ekranda gösterilebilir

,id,name
0,34382,mashed red potatoes with garlic and parmesan
1,106251,roasted cauliflower 16 roasted cloves of garlic
2,105726,kittencal s parmesan garlic bread or garlic toast
3,129040,peanut butter no bake cookies
4,100727,thick soft and chewy chocolate chip cookies
